In [ ]:
import torch
import numpy as np  # Import numpy
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification, Trainer, TrainingArguments
from transformers import DistilBertConfig
from transformers import DataCollatorForTokenClassification
from datasets import load_dataset
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import Trainer
from transformers import  get_cosine_schedule_with_warmup
from transformers import DistilBertModel
from torch.optim import AdamW
from collections import Counter

class CustomTrainer(Trainer):
    def create_optimizer_and_scheduler(self, num_training_steps: int):
        super().create_optimizer_and_scheduler(num_training_steps)  # Call to superclass method
        self.optimizer = AdamW(self.model.parameters(), betas=(0.9, 0.999), eps=1e-08, lr=5e-5)
        self.lr_scheduler = get_cosine_schedule_with_warmup(self.optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
        print(f'Optimizer: {self.optimizer}')  # Debugging line
        print(f'Scheduler: {self.lr_scheduler}')  # Debugging line

class CustomDistillBERTClass(torch.nn.Module):
    def __init__(self, num_labels, class_weights):
        super(CustomDistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-cased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, num_labels)
        self.class_weights = torch.tensor(class_weights).to('cuda')        
    
    def forward(self, input_ids, attention_mask, labels=None):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        
        # For token classification, we apply the classifier to each token in the sequence
        sequence_output = self.pre_classifier(hidden_state)
        sequence_output = torch.nn.ReLU()(sequence_output)
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        
        # If labels are provided, compute the loss as well
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights)     
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.classifier.out_features)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
        
        return logits if loss is None else (loss, logits)
        
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=2)

    # Remove ignored index (special tokens)
    true_labels = p.label_ids
    true_preds = [
        [id_to_tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, true_labels)
    ]
    true_labels = [
        [id_to_tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, true_labels)
    ]

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
        "accuracy": accuracy_score(true_labels, true_preds),
    }


# Load the dataset
dataset = load_dataset("wnut_17")
tag_to_id = dataset["train"].features["ner_tags"].feature.names
id_to_tag = {i: tag for i, tag in enumerate(tag_to_id)}

unique_tags = set()
for entry in dataset['train']['ner_tags']:
    unique_tags.update(entry)
print(f'Min tag: {min(unique_tags)}, Max tag: {max(unique_tags)}')

# Calculate class weights based on their frequency
counter = Counter({0: 59570, 9: 660, 7: 548, 10: 335, 5: 264, 8: 245, 1: 221, 4: 206, 12: 203, 6: 150, 11: 142, 3: 140, 2: 46})
total_count = sum(counter.values())
class_weights = [total_count / counter[i] for i in range(len(counter))]

# Normalize the class weights so they sum to 1
weight_sum = sum(class_weights)
normalized_class_weights = [w / weight_sum for w in class_weights]

# Multiply every label except the most common one by a factor of 10
most_common_label = counter.most_common(1)[0][0]
for i in range(len(class_weights)):
    if i != most_common_label:
        class_weights[i] *= 10

# Load the tokenizer and model using a DistilBERT checkpoint
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
config = DistilBertConfig.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))
model = CustomDistillBERTClass(num_labels=len(unique_tags), class_weights=normalized_class_weights)
model.to('cuda')


def tokenize_function(examples):
    encoding = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_attention_mask=True,
        return_tensors="pt"
    )
    encoding["labels"] = examples["ner_tags"]
    return encoding
    
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Prepare the data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir='./results',
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-05,  
    push_to_hub=False,  
    logging_first_step=True,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  
    greater_is_better=False,  
    warmup_steps=0,  
    save_total_limit=2,  
    seed=42
)

# Training
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

# Evaluation
results = trainer.evaluate()

# Print the results
print(results)

Min tag: 0, Max tag: 12
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 5e-05
    lr: 5e-05
    maximize: False
    weight_decay: 0.01
)
Scheduler: <torch.optim.lr_scheduler.LambdaLR object at 0x7ff7ced34430>


wandb: Currently logged in as: thejosephloy. Use `wandb login --relogin` to force relogin


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [1]:
import torch
import numpy as np  # Import numpy
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification, Trainer, TrainingArguments
from transformers import DistilBertConfig
from transformers import DataCollatorForTokenClassification
from datasets import load_dataset
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import Trainer
from transformers import  get_cosine_schedule_with_warmup
from transformers import DistilBertModel
from torch.optim import AdamW
from collections import Counter

class CustomTrainer(Trainer):
    def create_optimizer_and_scheduler(self, num_training_steps: int):
        super().create_optimizer_and_scheduler(num_training_steps)
        self.optimizer = AdamW(self.model.parameters(), betas=(0.9, 0.999), eps=1e-08, lr=2.44e-5)
        self.lr_scheduler = get_cosine_schedule_with_warmup(self.optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

class CustomDistillBERTClass(torch.nn.Module):
    def __init__(self, num_labels):
        super(CustomDistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-cased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        sequence_output = self.pre_classifier(hidden_state)
        sequence_output = torch.nn.ReLU()(sequence_output)
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()  # No weight argument
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.classifier.out_features)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
        
        return logits if loss is None else (loss, logits)
        
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=2)

    # Remove ignored index (special tokens)
    true_labels = p.label_ids
    true_preds = [
        [id_to_tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, true_labels)
    ]
    true_labels = [
        [id_to_tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, true_labels)
    ]

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
        "accuracy": accuracy_score(true_labels, true_preds),
    }


# Load the dataset
dataset = load_dataset("wnut_17")
tag_to_id = dataset["train"].features["ner_tags"].feature.names
id_to_tag = {i: tag for i, tag in enumerate(tag_to_id)}

unique_tags = set()
for entry in dataset['train']['ner_tags']:
    unique_tags.update(entry)
print(f'Min tag: {min(unique_tags)}, Max tag: {max(unique_tags)}')

# Load the tokenizer and model using a DistilBERT checkpoint
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
config = DistilBertConfig.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))
model = CustomDistillBERTClass(num_labels=len(unique_tags))
model.to('cuda')


def tokenize_function(examples):
    encoding = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_attention_mask=True,
        return_tensors="pt"
    )
    encoding["labels"] = examples["ner_tags"]
    return encoding
    
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Prepare the data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    output_dir='./results',
    num_train_epochs=7,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=2.44e-05,  
    push_to_hub=False,  
    logging_first_step=True,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  
    greater_is_better=False,  
    warmup_steps=0,  
    save_total_limit=2,  
    seed=42
)

# Training
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

# Evaluation
results = trainer.evaluate()

# Print the results
print(results)

Min tag: 0, Max tag: 12


wandb: Currently logged in as: thejosephloy. Use `wandb login --relogin` to force relogin


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.080700,0.610826,0.542484,0.099282,0.167846,0.926270
2,0.214100,0.488645,0.524390,0.205742,0.295533,0.931609
3,0.214100,0.564926,0.504559,0.198565,0.284979,0.930782
4,0.070900,0.760129,0.421875,0.193780,0.265574,0.929575
5,0.081000,0.775639,0.413793,0.229665,0.295385,0.928685
6,0.050000,0.803822,0.368522,0.229665,0.282977,0.927414
7,0.049500,0.827177,0.372816,0.229665,0.284234,0.927795


{'eval_loss': 0.488645076751709, 'eval_precision': 0.524390243902439, 'eval_recall': 0.20574162679425836, 'eval_f1': 0.29553264604810997, 'eval_accuracy': 0.9316087205237399, 'eval_runtime': 10.0002, 'eval_samples_per_second': 100.898, 'eval_steps_per_second': 100.898, 'epoch': 7.0}


In [ ]:
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import DistilBertTokenizerFast, DistilBertModel, Trainer, TrainingArguments, DataCollatorForTokenClassification
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
from collections import Counter
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup
import torch
import numpy as np

# Custom Trainer class
class CustomTrainer(Trainer):
    def create_optimizer_and_scheduler(self, num_training_steps: int):
        super().create_optimizer_and_scheduler(num_training_steps)
        self.optimizer = AdamW(self.model.parameters(), betas=(0.9, 0.999), eps=1e-08, lr=5e-5)
        self.lr_scheduler = get_cosine_schedule_with_warmup(self.optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Custom DistillBERT model class
class CustomDistillBERTClass(torch.nn.Module):
    def __init__(self, num_labels):
        super(CustomDistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-cased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        sequence_output = self.pre_classifier(hidden_state)
        sequence_output = torch.nn.ReLU()(sequence_output)
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.classifier.out_features)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
        
        return logits if loss is None else (loss, logits)

# Function to compute metrics
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=2)
    true_labels = p.label_ids
    true_preds = [
        [id_to_tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, true_labels)
    ]
    true_labels = [
        [id_to_tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, true_labels)
    ]

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
        "accuracy": accuracy_score(true_labels, true_preds),
    }

# Load the dataset
dataset = load_dataset("wnut_17")
tag_to_id = dataset["train"].features["ner_tags"].feature.names
id_to_tag = {i: tag for i, tag in enumerate(tag_to_id)}

# Upsampling smaller classes
counter = Counter()
for entry in dataset['train']['ner_tags']:
    counter.update(entry)

most_common_count = counter.most_common(1)[0][1]
upsampled_entries = []

for example in dataset['train']:
    tokens = example['tokens']
    ner_tags = example['ner_tags']
    example_counter = Counter(ner_tags)
    least_common_count = min([example_counter[tag] for tag in ner_tags if tag in example_counter])
    upsample_factor = most_common_count // least_common_count
    upsampled_entries.extend([example] * upsample_factor)

# Convert upsampled entries to a Dataset
upsampled_entries_dataset = Dataset.from_dict({
    'tokens': [entry['tokens'] for entry in upsampled_entries],
    'ner_tags': [entry['ner_tags'] for entry in upsampled_entries]
})

# Concatenate the original dataset with the upsampled dataset
upsampled_dataset = concatenate_datasets([dataset['train'], upsampled_entries_dataset])

# Tokenization function
def tokenize_function(examples):
    encoding = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_attention_mask=True,
        return_tensors="pt"
    )
    encoding["labels"] = examples["ner_tags"]
    return encoding

# Tokenize the upsampled dataset
tokenized_upsampled_dataset = upsampled_dataset.map(tokenize_function, batched=True)

# Prepare the data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir='./results',
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-05,
    push_to_hub=False,
    logging_first_step=True,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    warmup_steps=0,
    save_total_limit=2,
    seed=42
)

# Initialize the Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_upsampled_dataset,
    eval_dataset=dataset["validation"].map(tokenize_function, batched=True),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

# Print the results
print(results)

In [1]:
import optuna
import torch
import numpy as np  # Import numpy
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification, Trainer, TrainingArguments
from transformers import DistilBertConfig
from transformers import DataCollatorForTokenClassification
from datasets import load_dataset
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import Trainer
from transformers import  get_cosine_schedule_with_warmup
from transformers import DistilBertModel
from torch.optim import AdamW
from collections import Counter

class CustomTrainer(Trainer):
    def create_optimizer_and_scheduler(self, num_training_steps: int):
        super().create_optimizer_and_scheduler(num_training_steps)  # Call to superclass method
        self.optimizer = AdamW(self.model.parameters(), betas=(0.9, 0.999), eps=1e-08, lr=5e-5)
        self.lr_scheduler = get_cosine_schedule_with_warmup(self.optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
        print(f'Optimizer: {self.optimizer}')  # Debugging line
        print(f'Scheduler: {self.lr_scheduler}')  # Debugging line

class CustomDistillBERTClass(torch.nn.Module):
    def __init__(self, num_labels):
        super(CustomDistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-cased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, num_labels)
          
    
    def forward(self, input_ids, attention_mask, labels=None):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        
        # For token classification, we apply the classifier to each token in the sequence
        sequence_output = self.pre_classifier(hidden_state)
        sequence_output = torch.nn.ReLU()(sequence_output)
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        
        # If labels are provided, compute the loss as well
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()     
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.classifier.out_features)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
        
        return logits if loss is None else (loss, logits)
        
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=2)

    # Remove ignored index (special tokens)
    true_labels = p.label_ids
    true_preds = [
        [id_to_tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, true_labels)
    ]
    true_labels = [
        [id_to_tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, true_labels)
    ]

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
        "accuracy": accuracy_score(true_labels, true_preds),
    }


# Load the dataset
dataset = load_dataset("wnut_17")
tag_to_id = dataset["train"].features["ner_tags"].feature.names
id_to_tag = {i: tag for i, tag in enumerate(tag_to_id)}

unique_tags = set()
for entry in dataset['train']['ner_tags']:
    unique_tags.update(entry)
print(f'Min tag: {min(unique_tags)}, Max tag: {max(unique_tags)}')



# Load the tokenizer and model using a DistilBERT checkpoint
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
config = DistilBertConfig.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))
model = CustomDistillBERTClass(num_labels=len(unique_tags))
model.to('cuda')


def tokenize_function(examples):
    encoding = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_attention_mask=True,
        return_tensors="pt"
    )
    encoding["labels"] = examples["ner_tags"]
    return encoding
    
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Prepare the data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir='./results',
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-05,  
    push_to_hub=False,  
    logging_first_step=True,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  
    greater_is_better=False,  
    warmup_steps=0,  
    save_total_limit=2,  
    seed=42
)

# Training
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

def objective(trial):
    # Hyperparameters to be tuned
    lr = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    dropout = trial.suggest_float("dropout", 0.1, 0.3, step=0.1)

    # Initialize custom model with hyperparameters and other static settings
    num_labels = len(tag_to_id)
    model = CustomDistillBERTClass(num_labels=num_labels)
    model.dropout.p = dropout  # Set dropout rate
    model.to('cuda')  # Make sure the model is on the same device as the data

    # Initialize the Trainer with hyperparameters
    custom_trainer = CustomTrainer(
        model=model,
        args=TrainingArguments(
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            output_dir='./results',
            num_train_epochs=5,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            logging_dir='./logs',
            logging_steps=10,
            learning_rate=lr,  # Set learning rate
            push_to_hub=False,
            logging_first_step=True,
            load_best_model_at_end=True,
            metric_for_best_model='eval_loss',
            greater_is_better=False,
            warmup_steps=0,
            save_total_limit=2,
            seed=42
        ),
        train_dataset=tokenized_dataset["train"],  # Corrected dataset
        eval_dataset=tokenized_dataset["validation"],  # Corrected dataset
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    custom_trainer.train()

    # Evaluate the model
    results = custom_trainer.evaluate()

    return results["eval_loss"]  # Objective metric to be minimized

# Initialize Optuna study
study = optuna.create_study(direction="minimize")  # Minimize evaluation loss
study.optimize(objective, n_trials=50)  # Number of trials

# Access the optimal hyperparameters
best_params = study.best_params
best_value = study.best_value

print(f"Best parameters: {best_params}")
print(f"Best evaluation loss: {best_value}")

Min tag: 0, Max tag: 12


[I 2023-10-05 02:06:08,403] A new study created in memory with name: no-name-368c5a93-e58e-4077-98f9-2555584bce6f


Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 5e-05
    lr: 5e-05
    maximize: False
    weight_decay: 0.01
)
Scheduler: <torch.optim.lr_scheduler.LambdaLR object at 0x7fba1a7075e0>


wandb: Currently logged in as: thejosephloy. Use `wandb login --relogin` to force relogin


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 5e-05
    lr: 5e-05
    maximize: False
    weight_decay: 0.01
)
Scheduler: <torch.optim.lr_scheduler.LambdaLR object at 0x7fb9e4269120>


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.291200,0.437413,0.000000,0.000000,0.000000,0.920549
2,0.194300,0.361094,0.315789,0.028708,0.052632,0.922774
3,0.182400,0.356148,0.533333,0.076555,0.133891,0.925570
4,0.125100,0.383346,0.447099,0.156699,0.232064,0.927986
5,0.162100,0.383473,0.407311,0.186603,0.255947,0.927541


/home/thejoey/.local/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[I 2023-10-05 08:07:31,491] Trial 49 finished with value: 0.3561481833457947 and parameters: {'learning_rate': 2.8119082565489062e-05, 'dropout': 0.1}. Best is trial 9 with value: 0.3561481833457947.


Best parameters: {'learning_rate': 2.436768325486564e-05, 'dropout': 0.1}
Best evaluation loss: 0.3561481833457947
